In [2]:

import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
import cv2
import numpy as np

# API configuration
API_KEY = "###"  # Replace this with your Perenual API key
API_URL = "https://perenual.com/api/v2/plants"

# Create the main window
root = tk.Tk()
root.title("Plant Indicator App")

# Create a notebook (tabs) widget
notebook = ttk.Notebook(root)

# Create two frames (one for data entry, one for plant status)
frame1 = ttk.Frame(notebook)
frame2 = ttk.Frame(notebook)
frame3 = ttk.Frame(notebook)

# Add the frames to the notebook
notebook.add(frame1, text="Plant Input")
notebook.add(frame2, text="Plant Status")
notebook.add(frame3, text="Photo Recognition")
notebook.pack(expand=1, fill="both")

# Frame 1: Plant Input
plant_name_label = ttk.Label(frame1, text="Plant Name:")
plant_name_label.grid(row=0, column=0, padx=10, pady=10)

plant_name_entry = ttk.Entry(frame1)
plant_name_entry.grid(row=0, column=1, padx=10, pady=10)

soil_moisture_label = ttk.Label(frame1, text="Soil Moisture Level (0-100):")
soil_moisture_label.grid(row=1, column=0, padx=10, pady=10)

soil_moisture_entry = ttk.Entry(frame1)
soil_moisture_entry.grid(row=1, column=1, padx=10, pady=10)

light_level_label = ttk.Label(frame1, text="Light Level (0-100):")
light_level_label.grid(row=2, column=0, padx=10, pady=10)

light_level_entry = ttk.Entry(frame1)
light_level_entry.grid(row=2, column=1, padx=10, pady=10)

# Button to submit plant data
def submit_plant_data():
    plant_name = plant_name_entry.get()
    soil_moisture = soil_moisture_entry.get()
    light_level = light_level_entry.get()
    print(f"Plant: {plant_name}, Soil Moisture: {soil_moisture}, Light: {light_level}")
    status_label['text'] = f"Submitted data for {plant_name}!"

submit_button = ttk.Button(frame1, text="Submit", command=submit_plant_data)
submit_button.grid(row=3, column=1, padx=10, pady=10)

status_label = ttk.Label(frame1, text="")
status_label.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

# Button to check plant health
def check_plant_health():
    soil_moisture = int(soil_moisture_entry.get())
    if soil_moisture < 30:
        status_label['text'] = "Warning: Low soil moisture! Water the plant."
    else:
        status_label['text'] = "Soil moisture level is good."

check_button = ttk.Button(frame1, text="Check Plant Health", command=check_plant_health)
check_button.grid(row=5, column=1, padx=10, pady=10)

# Frame 2: Plant Status
status_title = ttk.Label(frame2, text="Current Plant Status:")
status_title.grid(row=0, column=0, padx=10, pady=10)

plant_status = tk.Text(frame2, height=10, width=40)
plant_status.grid(row=1, column=0, padx=10, pady=10)

def update_plant_status():
    plant_name = plant_name_entry.get()
    soil_moisture = soil_moisture_entry.get()
    light_level = light_level_entry.get()
    status_message = f"Plant: {plant_name}\nSoil Moisture: {soil_moisture}\nLight Level: {light_level}\n"
    plant_status.insert(tk.END, status_message)

update_button = ttk.Button(frame2, text="Update Status", command=update_plant_status)
update_button.grid(row=2, column=0, padx=10, pady=10)

# Frame 3: Photo Recognition
photo_label = ttk.Label(frame3, text="Upload a Plant Image:")
photo_label.grid(row=0, column=0, padx=10, pady=10)

# Label to display the selected image
image_label = tk.Label(frame3)
image_label.grid(row=1, column=0, padx=10, pady=10, columnspan=2)

def open_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        # Open the image with Pillow and display it
        img = Image.open(file_path)
        img.thumbnail((300, 300))  # Resize for display purposes
        img_tk = ImageTk.PhotoImage(img)
        image_label.config(image=img_tk)
        image_label.image = img_tk  # Keep a reference

        # Optionally, process the image with OpenCV
        process_image(file_path)

upload_button = ttk.Button(frame3, text="Upload Image", command=open_image)
upload_button.grid(row=0, column=1, padx=10, pady=10)

# Function to process the image with OpenCV
def process_image(image_path):
    # Load the image using OpenCV
    img = cv2.imread(image_path)

    # Example: Convert the image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Display the processed image in a new window
    cv2.imshow('Processed Image - Grayscale', gray_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
# Function to call the Perenual API
def recognize_plant(image_path):
    # Open the image file in binary mode
    with open(image_path, "rb") as image_file:
        # Prepare the API request
        files = {"image": image_file}
        params = {"api_key": API_KEY}
        
        # Send the request to Perenual API
        response = requests.post(f"{API_URL}/identify", files=files, params=params)
        
        if response.status_code == 200:
            data = response.json()
            if data and data.get('data'):
                # Extract plant info from the response
                plant_name = data['data'][0]['common_name']  # Example field: common name
                scientific_name = data['data'][0]['scientific_name']
                family = data['data'][0]['family']
                
                # Display the plant information in the GUI
                plant_info = f"Plant Name: {plant_name}\nScientific Name: {scientific_name}\nFamily: {family}"
                plant_info_label.config(text=plant_info)
            else:
                plant_info_label.config(text="No plant recognized.")
        else:
            plant_info_label.config(text=f"Error: {response.status_code}")

# Button to start processing the image
process_button = ttk.Button(frame3, text="Process Image", command=lambda: process_image(image_label.image))
process_button.grid(row=2, column=0, columnspan=2, padx=10, pady=10)

# Run the application
root.mainloop()

: 